# Tile masking
Masking is a common operation to use when performing analysis across tiles. Tile masking allows one to compare two tiles, a value tile and a masking tile. The masking tile can be thought of as being placed on top of the other tile, obscuring certain values. Masking cells set to NODATA are opaque, and cells that are anything else are transparent. In this way, value cells that correspond to masking cells that are NODATA become NODATA and value cells corresponding to other values are untouched.

Initial setup:

In [2]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession
from pathlib import Path

spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()

resource_dir = Path('./samples').resolve()

def readTiff(name):
    return resource_dir.joinpath(filenamePattern.format(name)).as_uri()

filenamePattern = "L8-B{}-Elkton-VA.tiff"
bandNumbers = range(1, 5)
bandColNames = list(map(lambda n: 'band_{}'.format(n), bandNumbers))

In [3]:
from functools import reduce
joinedRF = reduce(lambda rf1, rf2: rf1.asRF().spatialJoin(rf2.drop('bounds').drop('metadata')),
                  map(lambda bf: spark.read.geotiff(bf[1]) \
                      .withColumnRenamed('tile', 'band_{}'.format(bf[0])),
                  map(lambda b: (b, readTiff(b)), bandNumbers)))

In [4]:
joinedRF.select(cellType("band_1")).show()

+--------------------------+
|celltypeexpression(band_1)|
+--------------------------+
|                 uint16raw|
+--------------------------+



Because the tile has a cellType of raw, there is not a NODATA value. One possible solution is to convert the CellType to something that does support NODATA values. This transformation is wrapped into a threshold function that creates a new tile based on whether the cell values in an input tile are higher than a certain number.

In [5]:
# Need threshold function
def threshold(tile, thresh):
    tile.convert(IntConstantNoDataCellType).mapInt(lambda x: 1 if x > thresh else NODATA)

First, we create a masking tile based on a previous one (in this case, band_1 of the original tiff). This tile is full of either NODATA cells or the unchanged cell values corresponsing to whether the value in the input cell is greater than a threshold.

In [8]:
withMaskedTile = joinedRF.withColumn("maskTile", threshold(joinedRF("band_1"), 10500))

AttributeError: 'str' object has no attribute 'convert'

In [23]:
withMaskedTile.select(noDataCells("maskTile")).show()

+---------------------+
|noDataCells(maskTile)|
+---------------------+
|                28570|
+---------------------+



## Mask 
Mask is an operation that takes two tiles and eliminates data from one cell that corresponds to a value in the other. For instance, if a cell in the masking tile contains a NODATA, the corresponding cell in the other tile would be set to NODATA. If the masking cell contained anything else, the corresponding cell would have its value untouched.
### Inverse Mask
Inverse mask works in the opposite way; It doesn't touch the cells that are set to NODATA in the masking tile and all other cells become NODATA.

In [ ]:
val masked = withMaskedTile
  .withColumn("masked", mask(joinedRF("band_2"), withMaskedTile("maskTile"))).asRF
val inversemasked = withMaskedTile
  .withColumn("inversemasked", inverseMask(joinedRF("band_2"), withMaskedTile("maskTile"))).asRF

In [ ]:
val maskRaster = masked.toRaster(masked("masked"), 466, 428)
val inverseMaskRaster = inversemasked.toRaster(inversemasked("inversemasked"), 466, 428)

val brownToGreen = ColorRamp(
  RGBA(166,97,26,255),
  RGBA(223,194,125,255),
  RGBA(245,245,245,255),
  RGBA(128,205,193,255),
  RGBA(1,133,113,255)
).stops(128)

val colors = ColorMap.fromQuantileBreaks(maskRaster.tile.histogramDouble(), brownToGreen)

maskRaster.tile.color(colors).renderPng().write("mask.png")
inverseMaskRaster.tile.color(colors).renderPng().write("inverseMask.png")

Pictures go here